## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-15-20-37-41 +0000,nypost,Epstein called then-Prince Andrew ‘great fun’ ...,https://nypost.com/2025/11/15/world-news/epste...
1,2025-11-15-20-30-20 +0000,nypost,Victim stabbed twice in Times Square in broad ...,https://nypost.com/2025/11/15/us-news/victim-s...
2,2025-11-15-20-30-00 +0000,wsj,"News Quiz for Nov. 15, 2025",https://www.wsj.com/lifestyle/news-quiz-for-no...
3,2025-11-15-20-29-19 +0000,nyt,Fired Scholars and Big Grants to Favored Proje...,https://www.nytimes.com/2025/11/15/arts/nation...
4,2025-11-15-20-26-30 +0000,bbc,Zelensky vows energy sector overhaul after $10...,https://www.bbc.com/news/articles/cvgwyyd8l58o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2347/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
34,trump,52
0,epstein,14
85,he,11
324,nyc,9
209,no,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
15,2025-11-15-19-24-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...,131
166,2025-11-15-01-02-00 +0000,wsj,Trump said he would ask the Justice Department...,https://www.wsj.com/politics/policy/trump-call...,124
74,2025-11-15-15-47-56 +0000,nypost,Trump calls Marjorie Taylor Greene ‘disgrace’ ...,https://nypost.com/2025/11/15/us-news/trump-ca...,104
89,2025-11-15-13-22-06 +0000,cbc,Trump calls it quits with Marjorie Taylor Gree...,https://www.cbc.ca/news/world/u-s-trump-greene...,103
241,2025-11-14-21-07-00 +0000,wsj,Trump said he is withdrawing the nomination of...,https://www.wsj.com/politics/policy/trump-with...,102


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
15,131,2025-11-15-19-24-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...
207,42,2025-11-14-22-26-25 +0000,nypost,Jack Schlossberg shamelessly invokes Kennedy n...,https://nypost.com/2025/11/14/us-news/jack-sch...
41,37,2025-11-15-17-54-19 +0000,wapo,Former Fed governor violated central bank’s tr...,https://www.washingtonpost.com/business/2025/1...
241,37,2025-11-14-21-07-00 +0000,wsj,Trump said he is withdrawing the nomination of...,https://www.wsj.com/politics/policy/trump-with...
67,33,2025-11-15-16-45-59 +0000,nypost,Miss Israel getting death threats after being ...,https://nypost.com/2025/11/15/world-news/miss-...
110,31,2025-11-15-10-56-49 +0000,cbc,Vatican returns Indigenous cultural items to C...,https://www.cbc.ca/news/vatican-returns-indige...
229,31,2025-11-14-21-47-00 +0000,wsj,The reopened federal government will on Thursd...,https://www.wsj.com/economy/jobs/september-job...
128,30,2025-11-15-10-00-00 +0000,wsj,Trump administration officials including Healt...,https://www.wsj.com/politics/policy/rfk-jr-fda...
136,28,2025-11-15-07-21-56 +0000,wapo,"ESPN blackout on YouTube TV ends, as Disney an...",https://www.washingtonpost.com/business/2025/1...
192,26,2025-11-14-22-56-00 +0000,wsj,"Trump moved to lower duties on beef, coffee an...",https://www.wsj.com/economy/trade/trump-implem...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
